In [12]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

    
from src.utils import read_netmhcpan_results, set_hla
a02 = read_netmhcpan_results('../mhc_out/hla_a0201.xls')
b07 = read_netmhcpan_results('../mhc_out/hla_b0702.xls')
#pep = pd.read_csv('../test_data/test.pep', header = None)
#pep.columns = ['Peptide']


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


FileNotFoundError: [Errno 2] No such file or directory: '../mhc_out/hla_a0201.xls'

In [16]:
df.query('Peptide=="Peptide"').index

Int64Index([6, 8, 10], dtype='int64')

In [18]:
df =pd.read_csv('../output/test_hum_chunk_0.txt')
df.drop(index=df.query('Peptide=="Peptide"').index).reset_index)

SyntaxError: unmatched ')' (Temp/ipykernel_10356/804049299.py, line 2)

# parsing and analysis src

In [110]:
display(pep.head(1))
display(a02.head(1))
display(b07.head(1))

,Peptide
0,MSSHEGGKK


base                     HLA-A02:01                                       \
   Pos    Peptide       ID       core      icore EL-score EL_Rank BA-score   
0    0  MSSHEGGKK  PEPLIST  MSSHEGGKK  MSSHEGGKK   0.0001    48.4   0.0261   

              end     
  BA_Rank     Ave NB  
0  64.259  0.0001  0

base                     HLA-B07:02                                        \
   Pos    Peptide       ID       core      icore EL-score  EL_Rank BA-score   
0    0  MSSHEGGKK  PEPLIST  MSSHEGGKK  MSSHEGGKK   0.0001  39.4286   0.0325   

               end     
   BA_Rank     Ave NB  
0  47.6798  0.0001  0

In [111]:
def set_hla(df):
    hla = [x for x in df.columns.get_level_values(0).unique() if 'hla' in x.lower()][0]
    df.columns = df.columns.get_level_values(1)
    df['HLA'] = hla
    return df
a02 = set_hla(a02)
b07 = set_hla(b07)

In [127]:
a02.melt(id_vars=['Peptide','HLA']).to_csv('test2.csv')

In [128]:
"HLA-A02:01,HLA-B07:02,HLA-A03:01,HLA-A24:02,HLA-A11:01,HLA-B15:01,HLA-B35:01,HLA-A02:06,HLA-B27:05,HLA-A01:01".split(',')

['HLA-A02:01',
 'HLA-B07:02',
 'HLA-A03:01',
 'HLA-A24:02',
 'HLA-A11:01',
 'HLA-B15:01',
 'HLA-B35:01',
 'HLA-A02:06',
 'HLA-B27:05',
 'HLA-A01:01']

In [112]:
# Example with 2 outputs: 
# Query with threshold (here = 10) and Melt to concatenate
a10=a02.query('EL_Rank < 50').melt(id_vars = ['Peptide', 'HLA'])
b10=b07.query('EL_Rank < 50').melt(id_vars = ['Peptide', 'HLA'])
concat = pd.concat([a10,b10])
concat.head(3)

,Peptide,HLA,variable,value
0,ALKQPKKQA,HLA-A02:01,Pos,0
1,ALKQPKKQA,HLA-A02:01,ID,PEPLIST
2,ALKQPKKQA,HLA-A02:01,core,ALKQPKKQA


In [104]:
x = 'EL_Rank'
thr = 10.0
b07.query(f'{x} < @thr')

,Pos,ID,core,icore,EL-score,EL_Rank,BA-score,BA_Rank,Ave,NB,HLA
Peptide,,,,,,,,,,,
HEGGKKKAL,0,PEPLIST,HEGGKKKAL,HEGGKKKAL,0.0153,4.1139,0.0825,15.053,0.0153,0,HLA-B07:02
ALKQPKKQA,0,PEPLIST,ALKQPKKQA,ALKQPKKQA,0.0131,4.4668,0.0791,15.966,0.0131,0,HLA-B07:02


In [92]:
# Then pivot, reconvert values to float
pivot=concat.pivot(index=['Peptide', 'HLA'], columns = ['variable'], values = 'value')
pivot.EL_Rank = pivot.EL_Rank.astype(float)

In [102]:
pivot

variable                 Ave BA-score  BA_Rank EL-score  EL_Rank       ID NB  \
Peptide   HLA                                                                  
ALKQPKKQA HLA-A02:01  0.0094   0.0503  40.9719   0.0094   6.9344  PEPLIST  0   
          HLA-B07:02  0.0131   0.0791   15.966   0.0131   4.4668  PEPLIST  0   
HEGGKKKAL HLA-B07:02  0.0153   0.0825   15.053   0.0153   4.1139  PEPLIST  0   

variable             Pos       core      icore  
Peptide   HLA                                   
ALKQPKKQA HLA-A02:01   0  ALKQPKKQA  ALKQPKKQA  
          HLA-B07:02   0  ALKQPKKQA  ALKQPKKQA  
HEGGKKKAL HLA-B07:02   0  HEGGKKKAL  HEGGKKKAL

In [182]:
# Groupby level 0 (peptide level) and keep the idxmin on rank on level1
mins=pivot.loc[pivot.groupby(level=0)['EL_Rank'].idxmin()]
mins

,variable,Ave,BA-score,BA_Rank,EL-score,EL_Rank,ID,NB,Pos,core,icore
Peptide,HLA,,,,,,,,,,
ALKQPKKQA,HLA-A02:01,0.0094,0.0503,40.9719,0.0094,6.9344,PEPLIST,0,0,ALKQPKKQA,ALKQPKKQA
HEGGKKKAL,HLA-B07:02,0.0153,0.0825,15.053,0.0153,4.1139,PEPLIST,0,0,HEGGKKKAL,HEGGKKKAL


In [120]:
mins.reset_index()

Index(['Peptide', 'HLA', 'Ave', 'BA-score', 'BA_Rank', 'EL-score', 'EL_Rank',
       'ID', 'NB', 'Pos', 'core', 'icore'],
      dtype='object', name='variable')

In [124]:
pep.merge(mins.reset_index(), left_on='Peptide', right_on='Peptide')

,Peptide,ID_x,start_pos,HLA,Ave,BA-score,BA_Rank,EL-score,EL_Rank,ID_y,NB,Pos,core,icore
0,HEGGKKKAL,3,15,HLA-B07:02,0.0153,0.0825,15.053,0.0153,4.1139,PEPLIST,0,0,HEGGKKKAL,HEGGKKKAL
1,ALKQPKKQA,10,22,HLA-A02:01,0.0094,0.0503,40.9719,0.0094,6.9344,PEPLIST,0,0,ALKQPKKQA,ALKQPKKQA


# 11mers

In [161]:
mhc = set_hla(read_netmhcpan_results('../11mers_out.xls'))
txt = pd.read_csv('../11mers_merged.txt')

In [174]:
pepfn = '../output/test_hum_chunk_0.pep'
dirname = os.path.dirname(pepfn)+'/'
xlss = [x for x in os.listdir(dirname) if pepfn.replace(dirname,'').replace('.pep','') in x and 'xls' in x]
xlss

['test_hum_chunk_0_hla_a0201.xls', 'test_hum_chunk_0_hla_b0702.xls']

# 8mers with splits

In [3]:
results = read_netmhcpan_results('../../test_output/8mers_netmhcpan_out.xls')
results

base                    HLA-A02:01                                       \
     Pos   Peptide       ID       core     icore EL-score  EL_Rank BA-score   
0      0  FHAISXDH  PEPLIST  FHA-ISXDH  FHAISXDH   0.0000  80.0000   0.0097   
1      0  HAISXDHF  PEPLIST  HAISXDH-F  HAISXDHF   0.0001  38.4000   0.0196   
2      0  AISXDHFG  PEPLIST  AISXDHFG-  AISXDHFG   0.0000  60.6250   0.0237   
3      0  ISXDHFGA  PEPLIST  ISXDH-FGA  ISXDHFGA   0.0001  36.2500   0.0540   
4      0  SXDHFGAI  PEPLIST  SXD-HFGAI  SXDHFGAI   0.0001  46.0000   0.0392   
..   ...       ...      ...        ...       ...      ...      ...      ...   
397    0  TAAQTSGL  PEPLIST  TAAQ-TSGL  TAAQTSGL   0.0001  38.4000   0.0289   
398    0  AAQTSGLP  PEPLIST  AAQTSGL-P  AAQTSGLP   0.0000  67.5000   0.0160   
399    0  AQTSGLPS  PEPLIST  AQTSGLPS-  AQTSGLPS   0.0001  46.2000   0.0299   
400    0  QTSGLPSS  PEPLIST  QT-SGLPSS  QTSGLPSS   0.0000  56.8182   0.0185   
401    0  TSGLPSSV  PEPLIST  TS-GLPSSV  TSGLPSSV   0.0003  29.6000   0.0379   

             HLA-B07:02                                                   end  \
     BA_Rank       core     icore EL-score  EL_Rank BA-score  BA_Rank     Ave   
0    91.5834  FHA-ISXDH  FHAISXDH   0.0000  56.6667   0.0201  71.2179  0.0000   
1    74.2892  HAISXDH-F  HAISXDHF   0.0016  12.3787   0.0542  26.4153  0.0008   
2    67.8077  AISXDHFG-  AISXDHFG   0.0000  80.0000   0.0146  84.5950  0.0000   
3    38.6242  ISXDHFG-A  ISXDHFGA   0.0000  55.0000   0.0278  55.4259  0.0001   
4    49.5241  SXD-HFGAI  SXDHFGAI   0.0003  25.3000   0.0508  28.6113  0.0002   
..       ...        ...       ...      ...      ...      ...      ...     ...   
397  60.5865  TAA-QTSGL  TAAQTSGL   0.0063   6.3698   0.1226   8.4632  0.0032   
398  80.3296  AAQTSGLP-  AAQTSGLP   0.0001  45.2500   0.0389  39.2879  0.0000   
399  59.4344  AQTSGLPS-  AQTSGLPS   0.0000  60.7143   0.0271  56.8187  0.0000   
400  76.1159  QTSGLPSS-  QTSGLPSS   0.0001  44.2500   0.0216  67.7837  0.0001   
401  50.8289  TSGLP-SSV  TSGLPSSV   0.0003  26.6154   0.0466  31.8029  0.0003   

        
    NB  
0    0  
1    0  
2    0  
3    0  
4    0  
..  ..  
397  0  
398  0  
399  0  
400  0  
401  0  

[402 rows x 17 columns]

In [11]:
results.loc[results[('base', 'Peptide')].duplicated()].sort_values(('base', 'Peptide'))

base                    HLA-A02:01                                        \
     Pos   Peptide       ID       core     icore EL-score   EL_Rank BA-score   
290    0  AAVREVYE  PEPLIST  AAVREVYE-  AAVREVYE   0.0000   68.3333   0.0164   
299    0  AGVKGKLG  PEPLIST  AGVKGKLG-  AGVKGKLG   0.0000   90.0000   0.0063   
291    0  AVREVYEE  PEPLIST  AVREVYEE-  AVREVYEE   0.0001   46.8000   0.0201   
317    0  DRKHRTYV  PEPLIST  DR-KHRTYV  DRKHRTYV   0.0000   78.3333   0.0088   
338    0  DSVNIGRK  PEPLIST  DSV-NIGRK  DSVNIGRK   0.0000   80.0000   0.0089   
335    0  DWEDSVNI  PEPLIST  D-WEDSVNI  DWEDSVNI   0.0001   49.5000   0.0237   
298    0  EAGVKGKL  PEPLIST  EA-GVKGKL  EAGVKGKL   0.0000   58.6364   0.0087   
337    0  EDSVNIGR  PEPLIST  EDS-VNIGR  EDSVNIGR   0.0000   85.0000   0.0070   
334    0  EDWEDSVN  PEPLIST  EDWEDSVN-  EDWEDSVN   0.0000   90.0000   0.0050   
297    0  EEAGVKGK  PEPLIST  EEAG-VKGK  EEAGVKGK   0.0000   78.3333   0.0054   
331    0  EILEDWED  PEPLIST  EILEDWED-  EILEDWED   0.0000   90.0000   0.0114   
294    0  EVYEEAGV  PEPLIST  EVYEE-AGV  EVYEEAGV   0.0007   20.5747   0.0507   
289    0  GAAVREVY  PEPLIST  GAA-VREVY  GAAVREVY   0.0000   56.8182   0.0144   
288    0  GGAAVREV  PEPLIST  GGA-AVREV  GGAAVREV   0.0001   39.7778   0.0260   
303    0  GKLGRLLG  PEPLIST  GKLGRLLG-  GKLGRLLG   0.0000   95.0000   0.0071   
306    0  GRLLGIFE  PEPLIST  GRLLGIFE-  GRLLGIFE   0.0000   80.0000   0.0155   
300    0  GVKGKLGR  PEPLIST  GV-KGKLGR  GVKGKLGR   0.0000   59.5455   0.0118   
320    0  HRTYVYVL  PEPLIST  HRT-YVYVL  HRTYVYVL   0.0001   39.6667   0.0401   
342    0  IGRKREWF  PEPLIST  IGRKREW-F  IGRKREWF   0.0000   85.0000   0.0118   
332    0  ILEDWEDS  PEPLIST  ILEDWEDS-  ILEDWEDS   0.0000   59.5455   0.0262   
302    0  KGKLGRLL  PEPLIST  KG-KLGRLL  KGKLGRLL   0.0000   55.0000   0.0178   
319    0  KHRTYVYV  PEPLIST  KHR-TYVYV  KHRTYVYV   0.0000   54.0000   0.0361   
304    0  KLGRLLGI  PEPLIST  KL-GRLLGI  KLGRLLGI   0.0122    6.1424   0.2409   
333    0  LEDWEDSV  PEPLIST  -LEDWEDSV  LEDWEDSV   0.0002   30.9545   0.0545   
305    0  LGRLLGIF  PEPLIST  -LGRLLGIF  LGRLLGIF   0.0000   76.6667   0.0263   
308    0  LLGIFEQN  PEPLIST  LLGIFEQN-  LLGIFEQN   0.0000   60.6250   0.0408   
327    0  LTVTEILE  PEPLIST  LTVTEILE-  LTVTEILE   0.0000   64.3750   0.0270   
341    0  NIGRKREW  PEPLIST  NIGRK-REW  NIGRKREW   0.0000   64.3750   0.0093   
316    0  QDRKHRTY  PEPLIST  -QDRKHRTY  QDRKHRTY   0.0000   60.0000   0.0082   
293    0  REVYEEAG  PEPLIST  REVYEEAG-  REVYEEAG   0.0000  100.0000   0.0056   
318    0  RKHRTYVY  PEPLIST  RKH-RTYVY  RKHRTYVY   0.0000   66.6667   0.0111   
307    0  RLLGIFEQ  PEPLIST  RLLGIFEQ-  RLLGIFEQ   0.0014   15.6311   0.1003   
321    0  RTYVYVLT  PEPLIST  RTY-VYVLT  RTYVYVLT   0.0002   32.7778   0.0724   
339    0  SVNIGRKR  PEPLIST  SVN-IGRKR  SVNIGRKR   0.0001   47.5000   0.0139   
330    0  TEILEDWE  PEPLIST  TEILEDWE-  TEILEDWE   0.0000   95.0000   0.0081   
328    0  TVTEILED  PEPLIST  TVTEILED-  TVTEILED   0.0000   64.3750   0.0156   
322    0  TYVYVLTV  PEPLIST  -TYVYVLTV  TYVYVLTV   0.0005   22.8154   0.1255   
301    0  VKGKLGRL  PEPLIST  -VKGKLGRL  VKGKLGRL   0.0000   60.6250   0.0118   
326    0  VLTVTEIL  PEPLIST  VLT-VTEIL  VLTVTEIL   0.0129    5.9564   0.1868   
340    0  VNIGRKRE  PEPLIST  -VNIGRKRE  VNIGRKRE   0.0000   95.0000   0.0045   
292    0  VREVYEEA  PEPLIST  -VREVYEEA  VREVYEEA   0.0000   54.3333   0.0194   
329    0  VTEILEDW  PEPLIST  VT-EILEDW  VTEILEDW   0.0000   53.6667   0.0169   
295    0  VYEEAGVK  PEPLIST  -VYEEAGVK  VYEEAGVK   0.0000   64.3750   0.0190   
324    0  VYVLTVTE  PEPLIST  -VYVLTVTE  VYVLTVTE   0.0000   73.7500   0.0209   
336    0  WEDSVNIG  PEPLIST  WEDSVNIG-  WEDSVNIG   0.0000   95.0000   0.0098   
296    0  YEEAGVKG  PEPLIST  YEEAGVKG-  YEEAGVKG   0.0000   95.0000   0.0069   
325    0  YVLTVTEI  PEPLIST  YVL-TVTEI  YVLTVTEI   0.0167    5.2287   0.2134   
323    0  YVYVLTVT  PEPLIST  YVY-VLTVT  YVYVLTVT   0.0006   21.6974   0.0976   

   